In [1]:
# import tensorflow as tf
# tf.enable_eager_execution()

In [2]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [3]:
import numpy as np

from stog.utils.params import Params
from stog.data.dataset_builder import dataset_from_params, iterator_from_params
from stog.data.vocabulary import Vocabulary
from stog.training.trainer import Trainer
from stog.data.dataset import Batch
from model.text_to_amr import TextToAMR

from tensorflow.keras.layers import Embedding, Input, Dense, Flatten, LSTM, concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.utils import plot_model
from tensorflow.keras.losses import MeanSquaredError

[2020-02-28 14:38:42,510 INFO] Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .


In [4]:
params = Params.from_file("../model/model_params.yaml")

/home/adylanrff/Documents/Kuliah/TA/amr_parser/stog/utils/params.py:104: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  dict_merge.dict_merge(params_dict, yaml.load(f))


In [5]:
data_params = params['data']
dataset = dataset_from_params(data_params)

[2020-02-28 14:38:44,070 INFO] Building train datasets ...
[2020-02-28 14:38:44,071 ERROR] Model name 'data/bert-base-cased/bert-base-cased-vocab.txt' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-cased, bert-base-chinese). We assumed 'data/bert-base-cased/bert-base-cased-vocab.txt' was a path or url but couldn't find any file associated to this path or url.
0it [00:00, ?it/s][2020-02-28 14:38:44,074 INFO] Reading instances from lines in file at: ../data/raw/amr.txt.features
[2020-02-28 14:38:44,147 INFO] POS tag coverage: 0.3087 (184/596)
40it [00:00, 536.15it/s]
[2020-02-28 14:38:44,149 INFO] Building dev datasets ...
[2020-02-28 14:38:44,149 ERROR] Model name 'data/bert-base-cased/bert-base-cased-vocab.txt' was not found in model name list (bert-base-uncased, bert-large-uncased, bert-base-cased, bert-large-cased, bert-base-multilingual-uncased, bert-base-multilingual-

In [6]:
train_data = dataset['train']
dev_data = dataset.get('dev')
test_data = dataset.get('test')

In [7]:
vocab_params = params.get('vocab', {})
vocab = Vocabulary.from_instances(instances=train_data, **vocab_params)
vocab.save_to_files("../data/processed/serialization")

dataset = Batch(train_data)
dataset.index_instances(vocab)
dataset.index_instances(vocab)

# print(dataset.as_tensor_dict()['src_tokens']['encoder_tokens'][1].numpy())
# print(dataset.as_tensor_dict()['tgt_tokens']['decoder_tokens'][1].numpy())
# print(dataset.get_padding_lengths())
# print(dataset.as_tensor_dict())

for key in dataset.as_tensor_dict():
    print(key)
    content = dataset.as_tensor_dict()[key]
    if isinstance(content, dict):
        for inner_key in content:
            print("  ", inner_key)

train_iterator, dev_iterater, test_iterater = iterator_from_params(vocab, data_params['iterator'])

train_dataset = Batch(train_data)
train_dataset.index_instances(vocab)

test_dataset = Batch(test_data)
test_dataset.index_instances(vocab)

train_dataset = train_dataset.as_tensor_dict()
test_dataset = test_dataset.as_tensor_dict()

[2020-02-28 14:38:44,292 INFO] Fitting token dictionary from dataset.
100%|██████████| 40/40 [00:00<00:00, 2188.64it/s]
[2020-02-28 14:38:44,315 WARNING] vocabulary serialization directory ../data/processed/serialization is not empty


src_tokens
   encoder_tokens
   encoder_characters
src_must_copy_tags
tgt_tokens
   decoder_tokens
   decoder_characters
src_pos_tags
tgt_pos_tags
tgt_copy_indices
tgt_copy_mask
tgt_copy_map
src_copy_indices
src_copy_map
head_tags
head_indices


In [8]:
print(vocab)

Vocabulary with namespaces:
 	Non Padded Namespaces: {'must_copy_tags', 'coref_tags'}
 	Namespace: encoder_token_ids, Size: 334 
 	Namespace: encoder_token_characters, Size: 39 
 	Namespace: decoder_token_ids, Size: 283 
 	Namespace: decoder_token_characters, Size: 65 
 	Namespace: pos_tags, Size: 16 
 	Namespace: head_tags, Size: 35 



###### Test Model

In [9]:
text_to_amr = TextToAMR(vocab)
encoder_input, decoder_input, _, _ = text_to_amr.prepare_input(train_dataset)

ENCODER_INPUT
bert_token: None
token_subword_index: None
token: (40, 25)
pos_tag: (40, 25)
must_copy_tag: (40, 25)
char: (40, 25, 14)

DECODER_INPUT
token: (40, 28)
pos_tag: (40, 28)
char: (40, 28, 17)
coref: (40, 28)

GENERATOR_INPUT
vocab_targets: (40, 28)
coref_targets: (40, 28)
coref_attention_maps: (40, 28, 29)
copy_targets: (40, 28)
copy_attention_maps: (40, 25, 27)

PARSER_INPUT
edge_heads: (40, 27)
edge_labels: (40, 27)
corefs: (40, 28)
mask: (40, 28)



In [10]:
model = text_to_amr.generate_model()
model.compile(optimizer='rmsprop', loss="mean_squared_error")

ENCODER OUTPUT (40, 25, 400)
CONTEXT_VECTOR:  (40, 25, 400)
ATTENTION OUTPUT:  (40, 25, 400)
ATTENTION WEIGHT:  (40, 40, 25, 1)


[2020-02-28 14:38:46,114 WARNING] Layer dense_4 is casting an input tensor from dtype float64 to the layer's dtype of float32, which is new behavior in TensorFlow 2.  The layer has dtype float32 because it's dtype defaults to floatx.

If you intended to run this layer in float32, you can safely ignore this warning. If in doubt, this warning is likely only an issue if you are porting a TensorFlow 1.X model to TensorFlow 2.

To change all layers to have dtype float64 by default, call `tf.keras.backend.set_floatx('float64')`. To change just this layer, pass dtype='float64' to the layer constructor. If you are the author of this layer, you can disable autocasting by passing autocast=False to the base Layer constructor.



CONTEXT_VECTOR:  (40, 25, 400)
ATTENTION OUTPUT:  (40, 25, 400)
ATTENTION WEIGHT:  (40, 40, 25, 1)
CONTEXT_VECTOR:  (40, 25, 400)
CONTEXT_VECTOR:  (40, 25, 400)
ATTENTION OUTPUT:  (40, 25, 400)
ATTENTION WEIGHT:  (40, 40, 25, 1)


ValueError: in converted code:

    /home/adylanrff/Documents/Kuliah/TA/amr_parser/model/attention.py:20 call  *
        score = self.V(tf.nn.tanh(
    /home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/base_layer.py:737 __call__
        self.name)
    /home/linuxbrew/.linuxbrew/opt/python/lib/python3.7/site-packages/tensorflow_core/python/keras/engine/input_spec.py:155 assert_input_compatibility
        ' input tensors. Inputs received: ' + str(inputs))

    ValueError: Layer dense_4 expects 1 inputs, but it received 2 input tensors. Inputs received: [<tf.Tensor 'bahdanau_attention_1_1/Identity:0' shape=(40, 25, 400) dtype=float32>, <tf.Tensor 'bahdanau_attention_1_1/Identity_1:0' shape=(40, 40, 25, 1) dtype=float32>]


In [ ]:
# model.summary()

In [ ]:
# plot_model(model)

In [ ]:
# token_encoder_input = encoder_input.get('token')
# pos_encoder_input = encoder_input.get('pos_tag')
# token_decoder_input = decoder_input.get('token')
# pos_decoder_input = decoder_input.get('pos_tag')

# model_input = [token_encoder_input, pos_encoder_input, token_decoder_input, pos_decoder_input]
# model.fit(model_input, [1 for _ in range(len(model_input[0]))], epochs=10)